In [1]:
import cmipld
import importlib
from collections import OrderedDict


repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

whoami = cmipld.reverse_mapping()[io_url]
print('-'*50)
print(f'Parsing repo: {whoami}')
print(f'Location: {repo_url}')
print(f'Github IO link: {io_url}')
print('-'*50)

'''
To keep this set of functions generic, we create separate files containing all relevant functions for a repository to be processed. 
When running this script we determine the parent repository, and consequently its prefix. This is then used to grab all relevant functions for processing. 

Example usage: getattr(this,'activity')({'activity':[{"label":1,"id":'one'}]})

'''
# Dynamically import the target module and assign it to a variable
module_path = f"cmipld.cvs.{whoami}" # go to CMIPLD/CVs/<value of whoami>.py for functions
this = importlib.import_module(module_path)



make test for url2io
--------------------------------------------------
Parsing repo: cmip6plus
Location: https://github.com/WCRP-CMIP/CMIP6Plus_CVs
Github IO link: https://wcrp-cmip.github.io/CMIP6Plus_CVs/
--------------------------------------------------


/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # Set up offline override for the current repository

# ldpath = cmipld.utils.git.ldpath()


# # create a temporary directory containing (a copy of) the current repository
# # localserver = LD_server(copy = [[ldpath,'current_repo']])
# repos= { 
#         'https://wcrp-cmip.github.io/WCRP-universe/': 'universal',
#         'https://wcrp-cmip.github.io/MIP-variables/': 'variables',
#         'https://wcrp-cmip.github.io/CMIP6Plus_CVs/': 'cmip6plus'
#         }

# localserver = LD_server(repos = repos.items(),copy = [[ldpath,whoami]],override='y')

# # localserver = LD_server(copy = [[ldpath,'cmip6plus']])


# localhost = localserver.start_server(8080)


In [3]:
# # 
# replace_loader(localhost,[list(i) for i in repos.items()])
# # 

In [35]:

CV = OrderedDict()
cmipld.processor.clear_cache()


In [5]:


for item in [
    'repo',
    'mip-era',
    'license',
    'activity',
    'drs',
    'product',
    'required-global-attributes',
    # 'tables',
    ]:
    
    print(item)
    # CV[item] = getattr(locals(),item)
    ld  = cmipld.processor.expand_document(f'{io_url}project/{item}.json', depth=3)[0]
    # functions may be autogenrated in (reponame).py under the CVs dir. 
    CV[item] = getattr(this,item)(ld)
    


repo
mip-era
license
activity
dict_keys(['@context', 'id', 'type', 'description', 'label', 'start', 'url'])
dict_keys(['@context', 'id', 'type', 'description', 'label', 'start', 'url'])
drs
product
required-global-attributes


In [6]:
# adding the index properties
print('index')
CV.update(cmipld.processor.get(f'{io_url}/project/index.json')[0]['index'])

In [7]:
print('make a file for relevant realms and relevant realms')

for universe_item in ['realm','frequency']:
    # 'grid-label''nominal'
    print(universe_item)
    path = f'{cmipld.mapping["universal"]}{universe_item}/graph.jsonld'
    ld  = cmipld.jsonld.compact(path,path)['@graph']
    CV[universe_item] = getattr(this,universe_item)(ld)

make a file for relevant realms and relevant realms
realm
frequency


In [ ]:
# path = f'{cmipld.mapping["universal"]}resolution/graph.jsonld'
# data  = cmipld.jsonld.compact(path,path)['@graph']

# CV['nominal_resolution'] = [f"{r['value']} {r['unit']}" for r in sorted(data, key=lambda x: float(x['value']) if x['value'].replace('.', '', 1).isdigit() else float('inf'))]

print('make a file for relevant nominal resolution')

In [37]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 

es_data = {}

for value in ['experiment', 'source']:
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}experiment/graph.jsonld') 
    result = es_data[value].frame({'@type':[f'wcrp:{value}']})
    
    # CV[value+'_id'] = getattr(this,value)(result)
    break


 Please use .iterative_frame() instead.


100%|██████████| 4/4 [00:00<00:00, 26.92it/s]


In [38]:
def get_entry(data,entry='label'):
    if isinstance(data,dict):
        if 'id' in data:
            return [data[entry]]
        else:
            data = list(data.values())
    return [i.get(entry) for i in data]

def name_entry(data,value = 'description', key='label'):
    if isinstance(data,list):
        return {entry[key]: entry[value] for entry in data}
    
    elif isinstance(data,dict):
        if 'id' in data:
            return {data[key]:data[value]}
        else:
            return {entry: data[entry][value] for entry in data}
        
        

data = result[::]
dummy = {}
for e in data:
    print(e.keys())
    for name in ['model_realms','activity','parent_experiment','parent_activity','sub_experiment']:
        print(name)
        print(e[name])
        e[name] = name_entry(e[name])
        
    dummy[e['label']] = e
    
print(dummy)


dict_keys(['id', 'type', 'activity', 'description', 'end', 'label', 'min_number_yrs_per_sim', 'model_realms', 'parent_activity', 'parent_experiment', 'start', 'sub_experiment', 'tier'])
model_realms
[{'id': 'universal:source-type/aogcm', 'type': ['wcrp:source_type', 'universal'], 'description': 'coupled atmosphere-ocean global climate model, additionally including explicit representation of at least the land and sea ice', 'is_required': True, 'label': 'AOGCM'}, {'id': 'universal:source-type/aer', 'type': ['wcrp:source_type', 'universal'], 'description': 'aerosol treatment in an atmospheric model where concentrations are calculated based on emissions, transformation, and removal processes (rather than being prescribed or omitted entirely)', 'is_required': False, 'label': 'AER'}, {'id': 'universal:source-type/chem', 'type': ['wcrp:source_type', 'universal'], 'description': 'chemistry treatment in an atmospheric model that calculates atmospheric oxidant concentrations (including at least 

In [39]:
from pprint import pprint 
pprint(dummy)

{'1pctCO2': {'activity': {'CMIP': 'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, '
                                  'esm-piControl, esm-historical, historical, '
                                  'and piControl experiments'},
             'description': 'DECK: 1pctCO2',
             'end': -999,
             'id': 'cmip6plus:experiment/1pctco2',
             'label': '1pctCO2',
             'min_number_yrs_per_sim': 150,
             'model_realms': {'AER': 'aerosol treatment in an atmospheric '
                                     'model where concentrations are '
                                     'calculated based on emissions, '
                                     'transformation, and removal processes '
                                     '(rather than being prescribed or omitted '
                                     'entirely)',
                              'AOGCM': 'coupled atmosphere-ocean global '
                                       'climate model, additionally including '

In [ ]:
# url = cmipld.processor.resolve_prefix(file)

# loads the experiments graph and relevant dependancies. 
cmipld.processor.clear_cache()

es_data = {}

for value in ['experiment', 'source']:
    es_data[value] = cmipld.processor.EmbeddedFrame(f'{io_url}experiment/graph.jsonld') 
    
    ct = es_data[value].context
    ct['model_realms'] = {'@context': 
   
        {
            "@vocab":"https://wcrp-cmip.github.io/WCRP-universe/source-type/",
            "@base":"https://wcrp-cmip.github.io/WCRP-universe/source-type/",
            # "is_required":{"@value":'hello'},
            
            },
'@container': '@set'
# "@type":"@id"
    }
    
    
    # result = es_data[value].frame({'@type':[f'wcrp:{value}'],'@id':f'{io_url}experiment/abrupt-4xco2','@context':[es_data[value].context ], 'model_realms':{"is_required":{"@embed": "@never"}}})
    
    # '@id':f'{io_url}experiment/abrupt-4xco2'
    result = es_data[value].frame({'@type':[f'wcrp:{value}'],'@id':f'{io_url}experiment/abrupt-4xco2','@context':ct, 'model_realms':{"is_required":{"@embed": "@never"}}})
    
    a = es_data[value]
    break

result

In [ ]:
CV

In [91]:
# e1 = exp[1]
# # del e1['id'], e1['type']
# for k,v in e1.items():
#     if isinstance(v,dict) and 'type' in v:
#         print(k)
#         e1[k] = v['label']
# e1


In [ ]:
ls 
